<img src = "https://github.com/VeryFatBoy/notebooks/blob/main/common/images/img_github_singlestore-jupyter_featured_2.png?raw=true">

<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">How to Build Local LLM Apps with Ollama, DeepSeek-R1 and SingleStore Kai</h1>
    </div>
</div>

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Create an environment variable to point to your SingleStore Kai instance, as follows:</p>
        <p>export KAI_URL="mongodb://admin:&lt;password&gt;@&lt;host&gt;:27017/?authMechanism=PLAIN&tls=true&loadBalanced=true"</p>
    </div>
</div>

In [1]:
!pip cache purge --quiet

In [2]:
!pip install langchain-community langchain-ollama ollama pymongo --quiet

In [3]:
import ollama
import os
import pymongo
import re

from langchain_community.vectorstores import MongoDBAtlasVectorSearch
from langchain_core.documents import Document
from langchain_ollama import OllamaEmbeddings

In [4]:
embedding_model = "all-minilm"

ollama.pull(embedding_model)

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [5]:
llm = "deepseek-r1:1.5b"

ollama.pull(llm)

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [6]:
connection_url_kai = os.getenv("KAI_URL")

In [7]:
kai_client = pymongo.MongoClient(connection_url_kai)
db = kai_client["langchain_demo"]
collection = db["langchain_docs"]

In [8]:
documents = [
    "Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels",
    "Llamas were first domesticated and used as pack animals 4,000 to 5,000 years ago in the Peruvian highlands",
    "Llamas can grow as much as 6 feet tall though the average llama between 5 feet 6 inches and 5 feet 9 inches tall",
    "Llamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their body weight",
    "Llamas are vegetarians and have very efficient digestive systems",
    "Llamas live to be about 20 years old, though some only live for 15 years and others live to be 30 years old"
]

embeddings = OllamaEmbeddings(
    model = embedding_model,
)

dimensions = len(embeddings.embed_query(documents[0]))

docs = [Document(text) for text in documents]

In [9]:
collection.create_index(
    [("embedding", "vector")],
    name = "vector_index",
    kaiIndexOptions = {
        "index_type": "AUTO",
        "metric_type": "EUCLIDEAN_DISTANCE",
        "dimensions": dimensions
    }
)

'vector_index'

In [10]:
docsearch = MongoDBAtlasVectorSearch.from_documents(
    docs,
    embeddings,
    collection = collection,
    index_name = "vector_index"
)

In [11]:
prompt = "What animals are llamas related to?"
docs = docsearch.similarity_search(prompt)
data = docs[0].page_content
print(data)

Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels


In [12]:
output = ollama.generate(
    model = llm,
    prompt = f"Using this data: {data}. Respond to this prompt: {prompt}."
)

content = output["response"]
remove_think_tags = True

if remove_think_tags:
    content = re.sub(r"<think>.*?</think>", "", content, flags = re.DOTALL)

print(content)



Llamas are tailless tail Animals belonging to the Largilphas family. They share a lineage with vicuñas, which are also part of the camelid family, indicating close relatedness. However, unlike vicuñas, llamas do not possess tails, distinguishing them from other tailless tail Animals like the wild boars.
